<a href="https://colab.research.google.com/github/adrian-pieroni/challenge2-data-science-LATAM/blob/main/challenge2_data_science_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción




In [5]:
import pandas as pd

In [8]:
# URL de la API que devuelve un JSON
url = '/content/TelecomX_Data.json'

In [12]:
# Leer directamente desde la URL
df = pd.read_json(url)

FileNotFoundError: File /content/TelecomX_Data.json does not exist

In [11]:
# Vista previa de los datos
print(df.head())

NameError: name 'df' is not defined

✅ Ver nombres de columnas:

In [ ]:
df.columns

✅ Ver los primeros registros del DataFrame:

In [ ]:
df.head()       # Muestra las primeras 5 filas


In [ ]:
df.tail()       # Muestra las últimas 5 filas

✅ Ver los tipos de datos de cada columna:

In [ ]:
df.dtypes

✅ Resumen completo del DataFrame:

In [ ]:
df.info()

✅ Ver un resumen estadístico de las columnas numéricas:

In [ ]:
df.describe(include='object')

#🔧 Transformación



#📊 Carga y análisis



#📄Informe final

